In [1]:
import polars as pl
import sys, os
from sippy import *
import numpy as np
from sippy import functionset as fset
from sippy import functionsetSIM as fsetSIM
import matplotlib.pyplot as plt
import pandas as pd
import control as cnt
import json
from datetime import datetime, timezone
import plotly.graph_objects as go
from plotly_resampler import FigureResampler, FigureWidgetResampler

In [2]:
data_path = '/home/alqua/data/data_vdfs'

In [3]:
start_filter_date = datetime(2024, 1, 9, 00, 00, 0, tzinfo=timezone.utc)
end_filter_date = datetime(2024, 2, 4, 00, 00, 0, tzinfo=timezone.utc)

In [4]:
p1_power = pl.read_parquet(data_path + '/' +'pump1_power_siso.par')
p3_power = pl.read_parquet(data_path + '/' +'pump3_power_siso.par')
p4_power = pl.read_parquet(data_path + '/' +'pump4_power_siso.par')
outflow_df = pl.read_parquet(data_path + '/' +'outflow_miso.par')
pressure_df = pl.read_parquet(data_path + '/' +'pressure_miso.par')


In [5]:
sampling_time = '15s'

p1_power = p1_power.filter(
                                 (pl.col('time') >= start_filter_date) &
                                   (pl.col('time')<= end_filter_date)
                                 )

p3_power = p3_power.filter(
                                 (pl.col('time') >= start_filter_date) &
                                   (pl.col('time')<= end_filter_date)
                                 )

p4_power = p4_power.filter(
                                 (pl.col('time') >= start_filter_date) &
                                   (pl.col('time')<= end_filter_date)
                                 )


pressure_df = pressure_df.filter(
                                 (pl.col('time') >= start_filter_date) &
                                   (pl.col('time')<= end_filter_date)
                                 )

In [6]:
sysid_df = outflow_df.join(p1_power, 
                left_on='time', 
                right_on='time').join(p3_power, 
                left_on='time', 
                right_on='time').join(
                p4_power, 
                left_on='time', 
                right_on='time').join(pressure_df, 
                left_on='time', 
                right_on='time').upsample(
                    time_column='time', 
                    every=sampling_time, 
                    maintain_order=True).group_by_dynamic(
                        'time', 
                        every=sampling_time, 
                        ).agg(pl.all().mean())

In [7]:
features_cols = {
    'time': 'time_utc',  
    'level': 'h',
    'outflow': 'qout',
    'pump1_speed': 'p1_rpm',
    'pump4_speed': 'p4_rpm',
    'pump3_speed': 'p3_rpm',
    'pump1_power': 'p1_power',
    'pump3_power': 'p3_power',
    'pump4_power': 'p4_power',
    'pressure': 'pressure',
}

sysid_df = (
    sysid_df
    .select(features_cols.keys())
    .rename(features_cols)
    .filter(
        (pl.col("p1_rpm") > 0) |
        (pl.col("p3_rpm") > 0) |
        (pl.col("p4_rpm") > 0)
    )
)




# Exclude rows where all p1_rpm, p3_rpm, p4_rpm are greater than zero
sysid_df = sysid_df.with_columns((
    pl.when(pl.col("p1_rpm") > 0).then(pl.col("qout")).otherwise(0).alias("qout_p1"), 
    pl.when(pl.col("p3_rpm") > 0).then(pl.col("qout")).otherwise(0).alias("qout_p3"),
    pl.when(pl.col("p4_rpm") > 0).then(pl.col("qout")).otherwise(0).alias("qout_p4")
)).filter(~(
    (pl.col("p1_rpm") > 0) &
    (pl.col("p3_rpm") > 0) &
    (pl.col("p4_rpm") > 0)
))

In [8]:
from sippy import functionset as fset

def identify_system(df, u_col, y_col, test_size=0.6, na=1, nb=2, theta=0, dt=None, predict_test =False, nsteps_ahead=1, plot_results=False, save_tf=False):

    selected_data = df.select([u_col, y_col])
    split_point = int(len(selected_data) * test_size)
    train_df = selected_data.head(split_point)
    test_df = selected_data.tail(len(selected_data) - split_point) 
    


    u_train, u_test = train_df[u_col].to_numpy(), test_df[u_col].to_numpy()
    y_train, y_test = train_df[y_col].to_numpy(), test_df[y_col].to_numpy()
    
    na_ords = [na]         
    nb_ords = [[nb]]       
    theta = [[theta]] 

    id_ARX = system_identification(y_train, u_train, 'ARX', stab_cons=True, 
                                ARX_orders=[na_ords, nb_ords, theta], tsample=dt) 
    
    G = id_ARX.G  
    print(f"\nTransfer function from {u_col} to {y_col}:")
    print("==================")
    print(id_ARX.G)
    if save_tf: 
        tf_data = {
                    "u": u_col,
                    "y": y_col,
                    "na": na, 
                    "nb":nb,
                    "num": [round(x, 5) for x in id_ARX.NUMERATOR[0][0]],
                    "den": [round(x, 5) for x in id_ARX.DENOMINATOR[0][0][1:]],
                    "dt": dt
                }
        
        filename = f"tf_{u_col}_to_{y_col}.json"
        result_path = "results/"
        with open(result_path + filename, 'w') as f:
            json.dump(tf_data, f, indent=4)

    if predict_test: 
        t_test = np.arange(0, len(y_test)) * dt
        Yval = fset.validation(id_ARX, u_test, y_test, t_test, k=nsteps_ahead)
        if plot_results: 
            fig = FigureWidgetResampler(go.Figure())
            fig.update_layout(margin=dict(l=10, r=10, t=10, b=10))
            fig.add_trace(
                        go.Scattergl(
                            x=t_test,
                            y=y_test,
                            name=f'{y_col} (Predicted from {u_col})',  # Shows column names
                            showlegend=True,
                            mode='lines'
                        )
                    )
            fig.add_trace(
                        go.Scattergl(
                            x=t_test,
                            y=Yval.flatten(),
                            name=f'{y_col} (Predicted from {u_col})',  # Shows column names
                            showlegend=True,
                            mode='lines'
                        )
                    )
            fig.update_layout(height=200, template="plotly_dark")
            display(fig)
            
        
        return id_ARX, G, t_test, Yval

    
    else: 
        return id_ARX, G


In [9]:
columns = [
    ("p1_rpm", "qout_p1"),
    ("p1_rpm", "p1_power"),

    ("p3_rpm", "qout_p3"),
    ("p3_rpm", "p3_power"),

    ("p4_rpm", "qout_p4"),
    ("p4_rpm", "p4_power"),
]


for u_col, y_col in columns:
    print(f"Identifying model {u_col}, {y_col}")
    id_ARX, G, t_test, Yval = identify_system(
        df=sysid_df, 
        u_col=u_col,
        y_col=y_col,
        test_size=0.3, 
        na=1,
        nb=1,
        theta=0,
        dt=int(sampling_time[:2]),
        predict_test=True, 
        nsteps_ahead=1, 
        plot_results=True, 
        save_tf=True
    )
    # Process/save result as needed

Identifying model p1_rpm, qout_p1

Transfer function from p1_rpm to qout_p1:

 0.02131
----------
z - 0.9579

dt = 15



FigureWidgetResampler({
    'data': [{'mode': 'lines',
              'name': ('<b style="color:sandybrown">[R' ... ' style="color:#fc9944">~2k</i>'),
              'showlegend': True,
              'type': 'scattergl',
              'uid': 'bd5e2973-60aa-40d8-89d2-d329802c4d37',
              'x': array([      0,     840,    1965, ..., 1502910, 1504095, 1505565]),
              'y': array([426., 512., 401., ..., 457., 477.,   0.])},
             {'mode': 'lines',
              'name': ('<b style="color:sandybrown">[R' ... ' style="color:#fc9944">~2k</i>'),
              'showlegend': True,
              'type': 'scattergl',
              'uid': 'c22677b5-f541-4eed-967e-6074115e252c',
              'x': array([      0,      15,    1530, ..., 1502925, 1504110, 1505565]),
              'y': array([  0.        , 429.78154141, 497.02436814, ..., 459.47513903,
                          479.27153521,   0.        ])}],
    'layout': {'height': 200, 'margin': {'b': 10, 'l': 10, 'r': 10, 't': 10

Identifying model p1_rpm, p1_power

Transfer function from p1_rpm to p1_power:

0.003747
--------
z - 0.92

dt = 15



FigureWidgetResampler({
    'data': [{'mode': 'lines',
              'name': ('<b style="color:sandybrown">[R' ... ' style="color:#fc9944">~2k</i>'),
              'showlegend': True,
              'type': 'scattergl',
              'uid': '30ccc808-1410-4313-a058-7606aac1419b',
              'x': array([      0,     915,    1995, ..., 1503375, 1504110, 1505565]),
              'y': array([45.20000076, 51.29999924, 43.79999924, ..., 45.81999969, 47.68999863,
                           0.        ])},
             {'mode': 'lines',
              'name': ('<b style="color:sandybrown">[R' ... ' style="color:#fc9944">~2k</i>'),
              'showlegend': True,
              'type': 'scattergl',
              'uid': 'd3e18bad-2592-4f9d-ae6f-f2138bf0d46a',
              'x': array([      0,     105,    1605, ..., 1503390, 1504125, 1505565]),
              'y': array([ 0.        , 43.10739935, 49.68044217, ..., 45.97789728, 47.81076867,
                           0.        ])}],
    'layout':

Identifying model p3_rpm, qout_p3

Transfer function from p3_rpm to qout_p3:

 0.02136
----------
z - 0.9554

dt = 15



...

FigureWidgetResampler({
    'data': [{'mode': 'lines',
              'name': ('<b style="color:sandybrown">[R' ... ' style="color:#fc9944">~2k</i>'),
              'showlegend': True,
              'type': 'scattergl',
              'uid': '69f1198d-a37b-4786-bd1e-ec94380fe3f5',
              'x': array([      0,      15,    1530, ..., 1503300, 1504605, 1505565]),
              'y': array([ 0.        ,  0.        ,  0.        , ...,  0.        , 49.02000046,
                          42.11000061])},
             {'mode': 'lines',
              'name': ('<b style="color:sandybrown">[R' ... ' style="color:#fc9944">~2k</i>'),
              'showlegend': True,
              'type': 'scattergl',
              'uid': '21fa249e-886d-4dcb-ae55-76c7324fd93c',
              'x': array([      0,      15,    1530, ..., 1503300, 1504620, 1505565]),
              'y': array([ 0.        ,  0.        ,  0.        , ...,  0.        , 48.52029666,
                          40.55602472])}],
    'layout':

Identifying model p4_rpm, qout_p4

Transfer function from p4_rpm to qout_p4:

0.02154
--------
z - 0.96

dt = 15



FigureWidgetResampler({
    'data': [{'mode': 'lines',
              'name': ('<b style="color:sandybrown">[R' ... ' style="color:#fc9944">~2k</i>'),
              'showlegend': True,
              'type': 'scattergl',
              'uid': '0a8288f9-35c0-49e7-98c9-7d418b35bd05',
              'x': array([      0,      15,    1530, ..., 1501800, 1503300, 1505565]),
              'y': array([0., 0., 0., ..., 0., 0., 0.])},
             {'mode': 'lines',
              'name': ('<b style="color:sandybrown">[R' ... ' style="color:#fc9944">~2k</i>'),
              'showlegend': True,
              'type': 'scattergl',
              'uid': 'b5db2b78-3e04-4e06-a5d9-51b66f82ac03',
              'x': array([      0,      15,    1530, ..., 1501800, 1503300, 1505565]),
              'y': array([0., 0., 0., ..., 0., 0., 0.])}],
    'layout': {'height': 200, 'margin': {'b': 10, 'l': 10, 'r': 10, 't': 10}, 'template': '...'}
})

Identifying model p4_rpm, p4_power

Transfer function from p4_rpm to p4_power:

 0.004471
----------
z - 0.9017

dt = 15



FigureWidgetResampler({
    'data': [{'mode': 'lines',
              'name': ('<b style="color:sandybrown">[R' ... ' style="color:#fc9944">~2k</i>'),
              'showlegend': True,
              'type': 'scattergl',
              'uid': '13939e92-e62c-4576-86f6-c8bc6426163f',
              'x': array([      0,      15,    1530, ..., 1501800, 1503300, 1505565]),
              'y': array([0., 0., 0., ..., 0., 0., 0.])},
             {'mode': 'lines',
              'name': ('<b style="color:sandybrown">[R' ... ' style="color:#fc9944">~2k</i>'),
              'showlegend': True,
              'type': 'scattergl',
              'uid': 'bad8f9e1-3bee-467c-a6cf-5d558db3e357',
              'x': array([      0,      15,    1530, ..., 1501800, 1503300, 1505565]),
              'y': array([0., 0., 0., ..., 0., 0., 0.])}],
    'layout': {'height': 200, 'margin': {'b': 10, 'l': 10, 'r': 10, 't': 10}, 'template': '...'}
})